In [ ]:
import numpy as np 
import pandas as pd 
import os

## Fake or real by Anima_

First, this task we will use the "Cross-encoder" from transformers-based to did the task

https://sbert.net/examples/cross_encoder/applications/README.html

But it's too old for this task. We will use the SBert or Siamese (Bi-encoder) to do this task.

https://sbert.net/docs/sentence_transformer/training_overview.html

In [16]:
train_df = pd.read_csv("/kaggle/input/fake-or-real-the-impostor-hunt/data/train.csv")
train_df

,id,real_text_id
0,0,1
1,1,2
2,2,1
3,3,2
4,4,2
...,...,...
90,90,2
91,91,1
92,92,2
93,93,2


## Data Loader

In [22]:
import pandas as pd

directory = "/kaggle/input/fake-or-real-the-impostor-hunt/data/train"
i = 0

file1_list = []
file2_list = []
result_list = []

for name in os.listdir(directory):
    file1 = open(directory + "/" + name + "/file_1.txt")
    file1_txt = file1.read()
    file2 = open(directory + "/" + name + "/file_2.txt")
    file2_txt = file2.read()
    result = train_df.iloc[i]['real_text_id']
    file1_list.append(file1_txt)
    file2_list.append(file2_txt)
    result_list.append(result)
    i += 1

dict = {'file1_txt': file1_list, 'file2_txt': file2_list, 'label': result_list}
df = pd.DataFrame(dict)
df

In [25]:
import pandas as pd

directory = "/kaggle/input/fake-or-real-the-impostor-hunt/data/test"
i = 0

file1_list = []
file2_list = []
result_list = []

for name in os.listdir(directory):
    file1 = open(directory + "/" + name + "/file_1.txt")
    file1_txt = file1.read()
    file2 = open(directory + "/" + name + "/file_2.txt")
    file2_txt = file2.read()
    file1_list.append(file1_txt)
    file2_list.append(file2_txt)
    result_list.append(0)
    i += 1

dict = {'file1_txt': file1_list, 'file2_txt': file2_list, 'label': result_list}
test_df = pd.DataFrame(dict)
test_df

,file1_txt,file2_txt,label
0,Low or intermediate sized stars (like our sun)...,Stars with lower masses than our sun experienc...,0
1,"In the text, we examined the data-cube in the ...","In the text, we examined the data-cube in the ...",0
2,findpost\nNASA's powerful computer simulations...,Spectra used by astronomers across various fie...,0
3,A new generation of CCDs with improved red sen...,Fast forward: **the first hip-hop artist to re...,0
4,The astronomical community agrees it’s importa...,The astronomical community agrees that it is c...,0
...,...,...,...
1063,ESO doesn't have immediate funds allocated tow...,enic stargazing efforts are currently being co...,0
1064,The ESO-ESA Working Group on Fundamental Cosmo...,The ESO-ESA Working Group on Fundamental Cosmo...,0
1065,One of the key historical uses of Active Galac...,A key historical role of AGN (Active Galactic ...,0
1066,A call for proposals to conduct scientific ver...,## Operationally Challenged: The Saga of the ...,0


In [140]:
df.to_csv("train_df_cleaned.csv", index=False)
test_df.to_csv("test_df_cleaned.csv", index=False)

----

## Step 2 : SBERT Fine-tuning

In [156]:
from sentence_transformers import InputExample

# Create SBERT-style InputExamples
train_examples = [
    InputExample(texts=[row["file1_txt"], row["file2_txt"]], label=float(row["label"]))
    for _, row in df.iterrows()
]


In [142]:
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [153]:
from sentence_transformers import SentenceTransformer
import torch
import torch
import logging
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from torch.utils.data import DataLoader
import pandas as pd

word_model = models.Transformer("sentence-transformers/paraphrase-MiniLM-L6-v2", max_seq_length=256)
pooling_model = models.Pooling(word_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_model, pooling_model], device=device)

In [157]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

train_loss = losses.SoftmaxLoss(
    model=model,
    sentence_embedding_dimension=model.get_sentence_embedding_dimension(),
    num_labels=2  # binary classification
)

In [159]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    show_progress_bar=True
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [ ]:
# Predict similarity score
score = model.predict([["Some text from file1", "Some text from file2"]])

print(f"Similarity score: {score}")
